A sql practice by using  in-memory Sqlite database on a problem from Hackerrank.com.

Problem:

https://www.hackerrank.com/challenges/15-days-of-learning-sql?isFullScreen=true

sqlite initialization and in-memory db creation

In [1]:
import sqlite3 as sql


db = sql.connect("file::memory:?cache=shared", uri=True)


First Table creation

Hackers: The hacker_id is the id of the hacker, and name is the name of the Hacker.

In [2]:
db.execute("create table Hackers (hacker_id int,name text)")

Sample Data insertion for first table.

In [3]:
db.execute("INSERT INTO hackers VALUES(36396,'Frank');")
db.execute("INSERT INTO hackers VALUES(20703,'Angela');")
db.execute("INSERT INTO hackers VALUES(15758,'Rose');")
db.execute("INSERT INTO hackers VALUES(38289,'Patrick');")
db.execute("INSERT INTO hackers VALUES(44065,'Lisa');")
db.execute("INSERT INTO hackers VALUES(53473,'Kimberly');")
db.execute("INSERT INTO hackers VALUES(62529,'Bonnie');")
db.execute("INSERT INTO hackers VALUES(79722,'Michael');")

Second Table Creation:

Submissions: The submission_date is the date of the submission, submission_id is the id of the submission, hacker_id is the id of the hacker who made the submission, and score is the score of the submission. 

In [4]:
db.execute('CREATE TABLE submissions (submission_date date, submission_id integer, 	hacker_id integer, 	score integer)')

Sample Data insertion for Second table.

In [5]:
db.execute("INSERT INTO submissions VALUES('2016-03-01',8494,20703,0);")
db.execute("INSERT INTO submissions VALUES('2016-03-01',22403,53473,15);")
db.execute("INSERT INTO submissions VALUES('2016-03-01',23965,79722,60);")
db.execute("INSERT INTO submissions VALUES('2016-03-01',30173,36396,70);")
db.execute("INSERT INTO submissions VALUES('2016-03-02',34928,20703,0);")
db.execute("INSERT INTO submissions VALUES('2016-03-02',38740,15758,60);")
db.execute("INSERT INTO submissions VALUES('2016-03-02',42769,79722,60);")
db.execute("INSERT INTO submissions VALUES('2016-03-02',44364,79722,60);")
db.execute("INSERT INTO submissions VALUES('2016-03-03',45440,20703,0);")
db.execute("INSERT INTO submissions VALUES('2016-03-03',49050,36396,70);")
db.execute("INSERT INTO submissions VALUES('2016-03-03',50273,79722,5);")
db.execute("INSERT INTO submissions VALUES('2016-03-04',50344,20703,0);")
db.execute("INSERT INTO submissions VALUES('2016-03-04',51360,44065,90);")
db.execute("INSERT INTO submissions VALUES('2016-03-04',54404,53473,65);")
db.execute("INSERT INTO submissions VALUES('2016-03-04',61533,79722,45);")
db.execute("INSERT INTO submissions VALUES('2016-03-05',72852,20703,0);")
db.execute("INSERT INTO submissions VALUES('2016-03-05',74546,38289,0);")
db.execute("INSERT INTO submissions VALUES('2016-03-05',76487,62529,0);")
db.execute("INSERT INTO submissions VALUES('2016-03-05',82439,36396,10);")
db.execute("INSERT INTO submissions VALUES('2016-03-05',9006,36396,40);")
db.execute("INSERT INTO submissions VALUES('2016-03-06',90404,20703,0);")

Problem description :

On March 01, 2016 hackers 20703, 36396, 53473, and 79722 made submissions. There are 4 unique hackers who made at least one submission each day. As each hacker made one submission, 20703 is considered to be the hacker who made maximum number of submissions on this day. The name of the hacker is Angela.

On March 02, 2016 hackers 15758,20703 , and 79722 made submissions. Now 20703 and 79722 were the only ones to submit every day, so there are 2 unique hackers who made at least one submission each day. 79722 made 2 submissions, and name of the hacker is Michael.

On March 03, 2016 hackers 20703, 36396, and 79722 made submissions. Now and were the only ones, so there are unique hackers who made at least one submission each day. As each hacker made one submission so

is considered to be the hacker who made maximum number of submissions on this day. The name of the hacker is Angela.

On March 04, 2016 hackers 20703, 44065, 53473 and 79722 made submissions. Now and only submitted each day, so there are 2 unique hackers who made at least one submission each day. As each hacker made one submission so 20703 is considered to be the hacker who made maximum number of submissions on this day. The name of the hacker is Angela.

On March 05, 2016 hackers20703, 36396,38289 and 62529 made submissions. Now only 20703 submitted each day, so there is only 1 unique hacker who made at least one submission each day. 36396 made 2 submissions and name of the hacker is Frank.

On March 06, 2016 only 20703 made submission, so there is only unique hacker who made at least one submission each day. 20703 made 1 submission and name of the hacker is Angela.

Sample Output:

01/03/2016	4	20703	Angela 

02/03/2016	2	79722	Michael 

03/03/2016	2	20703	Angela 

04/03/2016	2	20703	Angela 

05/03/2016	1	36396	Frank 

06/03/2016	1	20703	Angela 


In [ ]:
import sqlite3 as sql


db = sql.connect("file::memory:?cache=shared", uri=True)


In [6]:
q = """
with Daily_Subs_Count_Per_Hacker as ( /* Hackers Daily submission Counts and total uniq hackers are counted.  */
	select hacker_id , submission_date , count(*) daily_sub_count, count(*) over(partition by submission_date) daily_uniq_hacker_count
	from submissions s 
	group by hacker_id, submission_date)
,Dist_Hackers_Day as ( /* Distinct Hackers and Submission Dates are selected.  */
	select distinct hacker_id, submission_date 
	from Daily_Subs_Count_Per_Hacker
	where submission_date between '2016-03-01' and '2016-03-15'
	order by hacker_id, submission_date asc)
,Dist_Days as (/*index numbres are created for each contest days */
	select ud.uniq_days, row_number() over(order by ud.uniq_days)  day_number
	from (select distinct submission_date as uniq_days 
			from Dist_Hackers_Day
		 ) ud)	
,Daily_Subs_Status as ( /* Difference between Index numbers of Contest days and a hackers' submission days checked 
						  if they were equal, means that daily submission has been made */
	select hacker_id, submission_date,  
	case 
			when dd.day_number  - row_number() over(partition by hacker_id   order by submission_date) = 0 
			then 1 
		else 0 
		end 
		dail_sub_check
	from Dist_Hackers_Day dhd,Dist_Days dd
	where dhd.submission_date = dd.uniq_days)
,Ranked as ( /* Hackers are ranked over daily submission counts  */
	select dss.submission_date , dss.hacker_id, dss.dail_sub_check, dsh.daily_sub_count , dsh.daily_uniq_hacker_count,  
		dense_rank () over(partition by dss.submission_date order by dsh.daily_sub_count desc, dsh.daily_sub_count DESC ) rank,
		sum(dail_sub_check) over(partition by dss.submission_date) total_consequtive_single_hacker_count
	from Daily_Subs_Status dss, Daily_Subs_Count_Per_Hacker dsh 
	where dss.hacker_id = dsh.hacker_id 
	and dss.submission_date = dsh.submission_date )
,results as( /*For equal ranks, the min hacker_id is selected */
	select r.submission_date, r.total_consequtive_single_hacker_count, min(r.hacker_id) hacker_id 
	from ranked r 
	where rank = 1
	group by r.submission_date, r.total_consequtive_single_hacker_count
	order by r.submission_date)
select r.submission_date , r.total_consequtive_single_hacker_count uniq_hacker, r.hacker_id ,h.name
from results r, hackers h 
where r.hacker_id = h.hacker_id 
order by r.submission_date ;
"""

Sample Output:

2016-03-01 4 20703 Angela

2016-03-02 2 79722 Michael

2016-03-03 2 20703 Angela

2016-03-04 2 20703 Angela

2016-03-05 1 36396 Frank

2016-03-06 1 20703 Angela

In [7]:
list(db.execute(q))

[('2016-03-01', 4, 20703, 'Angela'),
 ('2016-03-02', 2, 79722, 'Michael'),
 ('2016-03-03', 2, 20703, 'Angela'),
 ('2016-03-04', 2, 20703, 'Angela'),
 ('2016-03-05', 1, 36396, 'Frank'),
 ('2016-03-06', 1, 20703, 'Angela')]